## Imports

In [ ]:
import os
os.chdir("../")
%env CUDA_VISIBLE_DEVICES=0
%matplotlib inline

In [ ]:
import numpy as np                                                              
import matplotlib.pyplot as plt
import scipy
import tensorflow as tf                                                         
import data.data_selector as ds                                                   
import analysis.analysis_picker as ap
import utils.plot_functions as pf                                               

## Model parameters

In [ ]:
class analysis_params(object):
  def __init__(self):
    self.model_type = "lca_subspace"
    self.model_name = "lca_subspace_mnist"
    self.version = "0.0"
    self.save_info = "analysis_test_carlini_targeted"
    self.overwrite_analysis_log = False

# Computed params
params = analysis_params()
params.model_dir = (os.path.expanduser("~")+"/Work/Projects/"+params.model_name)

## Load analyzer

In [ ]:
analyzer = ap.get_analyzer(params.model_type)
analyzer.setup(params)
analyzer.setup_model(analyzer.model_params)
analyzer.load_analysis(save_info=params.save_info)
analyzer.model_name = params.model_name

### New Analysis

### MNIST

### VanHateren

## Construct analysis plots

In [ ]:
fig = pf.plot_loc_freq_summary(analyzer.bf_stats)
fig.savefig(analyzer.analysis_out_dir+"vis/location_frequency_centers.png")

In [ ]:
neuron_angles = analyzer.get_neuron_angles(analyzer.bf_stats)

In [ ]:
angle_hist_fig = pf.plot_weight_angle_histogram(neuron_angles[1], num_bins=50, angle_min=0, angle_max=180, figsize=(8,8))
angle_hist_fig.savefig(analyzer.analysis_out_dir+"/vis/neuron_angle_histogram.png")

In [ ]:
weight_indices = np.stack([np.array(id_list) for id_list in analyzer.model.module.group_ids], axis=0)
pooling_weights = np.stack([analyzer.evals["lca_subspace/weights/w:0"][:, id_list] for id_list in analyzer.model.module.group_ids], axis=1)

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib.ticker import FormatStrFormatter
from matplotlib.colors import LinearSegmentedColormap
from mpl_toolkits import axes_grid1
import utils.data_processing as dp
def plot_group_weights(weights, group_ids, title="", figsize=None,  save_filename=None):
  """
    weights: [np.ndarray] of shape [num_neurons, num_input_y, num_input_x]
    group_ids: [list of lists] containing ids for each group [[,]*neurons_per_group,]*num_groups
  """
  num_neurons = weights.shape[0]
  for weight_id in range(num_neurons):
    weights[weight_id,...] = weights[weight_id,...] - weights[weight_id,...].mean()
    weights[weight_id,...] = weights[weight_id,...] / (weights[weight_id,...].max()-weights[weight_id,...].min())
  vmin = np.min(weights)
  vmax = np.max(weights)
  indices = [idx for id_list in group_ids for idx in id_list]
  num_groups = len(group_ids)
  num_groups_x = int(np.floor(np.sqrt(num_groups)))
  num_groups_y = int(np.ceil(np.sqrt(num_groups)))
  num_neurons_per_group = len(group_ids[0])
  num_neurons_x = int(np.floor(np.sqrt(num_neurons_per_group)))
  num_neurons_y = int(np.ceil(np.sqrt(num_neurons_per_group)))
  outer_spacing = 0.20
  inner_spacing = 0.1
  fig = plt.figure(figsize=figsize)
  gs1 = gridspec.GridSpec(num_groups_y, num_groups_x,
    hspace=outer_spacing*num_groups_y/(num_groups_x+num_groups_y),
    wspace=outer_spacing*num_groups_x/(num_groups_x+num_groups_y))
  neuron_index = 0
  for group_plot_id in np.ndindex((num_groups_y, num_groups_x)):
    gs_inner = gridspec.GridSpecFromSubplotSpec(num_neurons_y, num_neurons_x, gs1[group_plot_id],
      hspace=inner_spacing*num_neurons_y/(num_neurons_x+num_neurons_y),
      wspace=inner_spacing*num_neurons_x/(num_neurons_x+num_neurons_y))
    for inner_plot_id in np.ndindex((num_neurons_y, num_neurons_x)):
      ax = pf.clear_axis(fig.add_subplot(gs_inner[inner_plot_id]))
      ax.set_aspect("equal")
      if neuron_index < num_neurons:
        ax.imshow(weights[indices[neuron_index], ...], cmap="Greys_r", vmin=vmin, vmax=vmax)
        neuron_index += 1
  fig.suptitle(title, y=0.9, x=0.5, fontsize=20)
  if save_filename is not None:
    fig.savefig(save_filename)
    plt.close(fig)
    return None
  plt.show()
  return fig

In [ ]:
weights = np.reshape(analyzer.evals["lca_subspace/weights/w:0"].T, [analyzer.model.params.num_neurons,
      int(np.sqrt(analyzer.model.params.num_pixels)), int(np.sqrt(analyzer.model.params.num_pixels))])
weight_fig = plot_group_weights(np.squeeze(weights), analyzer.model.module.group_ids,
  title="Dictionary", figsize=(18,18))
weight_fig.savefig(analyzer.analysis_out_dir+"/vis/"+"group_phi.png")